# Melon Texture Classifer 

## Read dataset 

In [53]:
from sklearn.datasets import load_files
from glob import glob
import numpy as np
from keras.utils import np_utils

def load_dataset(path):
    data = load_files(path)
    melon_filenames = data['filenames']
    melon_targets = np_utils.to_categorical(np.array(data['target']), 6)
    return melon_filenames, melon_targets

train_files, train_targets = load_dataset('MelonImages/train')
valid_files, valid_targets = load_dataset('MelonImages/valid')
test_files, test_targets = load_dataset('MelonImages/test')
melon_names = [item[20:-1] for item in sorted(glob('MelonImages/train/*/'))]

#print some statistics
print('There are %d total melon categories.' % len(melon_names))
print('There are %s total melon images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training melon images.' % len(train_files))
print('There are %d validation melon images.' % len(valid_files))
print('There are %d test melon images.'% len(test_files))

There are 6 total melon categories.
There are 117 total melon images.

There are 68 training melon images.
There are 23 validation melon images.
There are 26 test melon images.


In [48]:
len(valid_files)
valid_tensors = paths_to_tensor(valid_files)



  0%|          | 0/68 [00:00<?, ?it/s]

100%|██████████| 68/68 [00:00<00:00, 839.99it/s]

23

In [54]:
from keras.preprocessing import image
from tqdm import tqdm

def path_to_tensor(path):
    img = image.load_img(path, target_size=(128,128))
    tensor_3d = image.img_to_array(img)
    return np.expand_dims(tensor_3d, axis=0)

def paths_to_tensor(image_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(image_paths)]
    return np.vstack(list_of_tensors)

In [55]:
train_tensors = paths_to_tensor(train_files).astype('float32')
valid_tensors = paths_to_tensor(valid_files).astype('float32')
test_tensors = paths_to_tensor(test_files).astype('float32')



  0%|          | 0/68 [00:00<?, ?it/s]

100%|██████████| 68/68 [00:00<00:00, 938.99it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

100%|██████████| 23/23 [00:00<00:00, 1059.96it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

100%|██████████| 26/26 [00:00<00:00, 1001.10it/s]

In [196]:
import keras
MELON_SHAPE = (128,128,3)

frame_input = keras.layers.Input(MELON_SHAPE, name='input')

normalized = keras.layers.Lambda(lambda x: x/255.0)(frame_input)

conv_1 = keras.layers.convolutional.Conv2D(16, (8, 8), strides=(2,2), activation='relu', 
                                            kernel_initializer='glorot_normal')(normalized)

#max_pool_1 = keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid')(conv_1)

conv_2 = keras.layers.convolutional.Conv2D(32, (8, 8), strides=(2,2), activation='relu', 
                                            kernel_initializer='glorot_normal')(conv_1)

max_pool_2 = keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid')(conv_2)

flattened = keras.layers.core.Flatten()(max_pool_2)

#hidden_1 = keras.layers.Dense(576, activation='relu')(flattened)
#out_hidden_1 = keras.layers.Dropout(0.2)(hidden_1)

hidden_2 = keras.layers.Dense(128, activation='relu')(flattened)
out_hidden_2 = keras.layers.Dropout(0.2)(hidden_2)

net_output = keras.layers.Dense(6, activation='softmax')(out_hidden_2)

Model = keras.models.Model(inputs=frame_input, outputs=net_output)
#optimizer = keras.optimizers.RMSprop(lr=0.00001)
optimizer = keras.optimizers.Adam(lr=0.0001)
Model.compile(optimizer=optimizer, loss='categorical_crossentropy')
Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 128, 128, 3)       0         
_________________________________________________________________
lambda_44 (Lambda)           (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_87 (Conv2D)           (None, 61, 61, 16)        3088      
_________________________________________________________________
conv2d_88 (Conv2D)           (None, 27, 27, 32)        32800     
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_43 (Flatten)         (None, 5408)              0         
_________________________________________________________________
dense_94 (Dense)             (None, 128)               692352    
__________

In [175]:
from keras.callbacks import ModelCheckpoint

epochs = 130
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.keras.hdf5',
                              verbose=1, save_best_only=True)

Model.fit(train_tensors, train_targets,
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=17, callbacks=[checkpointer], verbose=1)

Train on 68 samples, validate on 23 samples
Epoch 1/130
68/68 [==============================] - 0s - loss: 0.3983 - val_loss: 0.8560
Epoch 2/130
68/68 [==============================] - 0s - loss: 0.3883 - val_loss: 0.8883
Epoch 3/130
68/68 [==============================] - 0s - loss: 0.3553 - val_loss: 1.1526
Epoch 4/130
68/68 [==============================] - 0s - loss: 0.4107 - val_loss: 1.0346
Epoch 5/130
68/68 [==============================] - 0s - loss: 0.3721 - val_loss: 1.0581
Epoch 6/130
68/68 [==============================] - 0s - loss: 0.3254 - val_loss: 0.8820
Epoch 7/130
68/68 [==============================] - 0s - loss: 0.3644 - val_loss: 0.9637
Epoch 8/130
68/68 [==============================] - 0s - loss: 0.3186 - val_loss: 1.0393
Epoch 9/130
68/68 [==============================] - 0s - loss: 0.2996 - val_loss: 0.9511
Epoch 10/130
68/68 [==============================] - 0s - loss: 0.2887 - val_loss: 0.9084
Epoch 11/130
68/68 [==============================] - 0

68/68 [==============================] - 0s - loss: 0.1549 - val_loss: 0.9065
Epoch 86/130
68/68 [==============================] - 0s - loss: 0.1485 - val_loss: 0.9845
Epoch 87/130
68/68 [==============================] - 0s - loss: 0.1786 - val_loss: 0.9748
Epoch 88/130
68/68 [==============================] - 0s - loss: 0.1938 - val_loss: 0.9353
Epoch 89/130
68/68 [==============================] - 0s - loss: 0.1411 - val_loss: 0.9533
Epoch 90/130
68/68 [==============================] - 0s - loss: 0.1654 - val_loss: 0.9769
Epoch 91/130
68/68 [==============================] - 0s - loss: 0.1462 - val_loss: 0.9205
Epoch 92/130
68/68 [==============================] - 0s - loss: 0.1407 - val_loss: 0.9626
Epoch 93/130
68/68 [==============================] - 0s - loss: 0.1402 - val_loss: 0.9325
Epoch 94/130
68/68 [==============================] - 0s - loss: 0.1780 - val_loss: 0.9197
Epoch 95/130
68/68 [==============================] - 0s - loss: 0.1671 - val_loss: 0.9514
Epoch 96/130

68/68 [==============================] - 0s - loss: 0.1322 - val_loss: 0.9247
Epoch 128/130
68/68 [==============================] - 0s - loss: 0.1160 - val_loss: 0.8887
Epoch 129/130
68/68 [==============================] - 0s - loss: 0.0893 - val_loss: 0.9596
Epoch 130/130
68/68 [==============================] - 0s - loss: 0.1139 - val_loss: 0.9862


In [176]:
Model.load_weights('saved_models/weights.best.keras.hdf5')

melon_predictions = [np.argmax(Model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

test_accuracy = 100*np.sum(np.array(melon_predictions)==np.argmax(test_targets, axis=1))/len(melon_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 57.6923%


In [187]:
Model.load_weights('saved_models/weights.best.keras.hdf5')

all_tensors = np.vstack([train_tensors, valid_tensors, test_tensors])
all_targets = np.vstack([train_targets, valid_targets, test_targets])

melon_predictions = [np.argmax(Model.predict(np.expand_dims(tensor, axis=0))) for tensor in all_tensors]

test_accuracy = 100*np.sum(np.array(melon_predictions)==np.argmax(all_targets, axis=1))/len(melon_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 81.1966%


In [188]:
from keras.callbacks import ModelCheckpoint

epochs = 130
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.keras.all.hdf5',
                              verbose=1, save_best_only=True)

Model.fit(all_tensors, all_targets,
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=17, callbacks=[checkpointer], verbose=1)

Train on 117 samples, validate on 23 samples
Epoch 1/130
117/117 [==============================] - 1s - loss: 0.6492 - val_loss: 1.0188
Epoch 2/130
117/117 [==============================] - 1s - loss: 0.7190 - val_loss: 0.9161
Epoch 3/130
117/117 [==============================] - 1s - loss: 0.9278 - val_loss: 0.9097
Epoch 4/130
117/117 [==============================] - 1s - loss: 0.6728 - val_loss: 0.9478
Epoch 5/130
117/117 [==============================] - 1s - loss: 0.7336 - val_loss: 1.1930
Epoch 6/130
117/117 [==============================] - 1s - loss: 0.6299 - val_loss: 0.7908
Epoch 7/130
117/117 [==============================] - 1s - loss: 0.5926 - val_loss: 0.8063
Epoch 8/130
117/117 [==============================] - 1s - loss: 0.5101 - val_loss: 0.6904
Epoch 9/130
117/117 [==============================] - 1s - loss: 0.5777 - val_loss: 0.7557
Epoch 10/130
117/117 [==============================] - 1s - loss: 0.5781 - val_loss: 0.7308
Epoch 11/130
117/117 [============

117/117 [==============================] - 1s - loss: 0.3666 - val_loss: 0.4820
Epoch 38/130
117/117 [==============================] - 1s - loss: 0.4541 - val_loss: 0.5516
Epoch 39/130
117/117 [==============================] - 1s - loss: 0.4944 - val_loss: 0.5670
Epoch 40/130
117/117 [==============================] - 1s - loss: 0.4008 - val_loss: 0.5519
Epoch 41/130
117/117 [==============================] - 1s - loss: 0.3925 - val_loss: 0.4690
Epoch 42/130
117/117 [==============================] - 1s - loss: 0.3485 - val_loss: 0.4196
Epoch 43/130
117/117 [==============================] - 1s - loss: 0.3258 - val_loss: 0.4251
Epoch 44/130
117/117 [==============================] - 1s - loss: 0.3168 - val_loss: 0.4113
Epoch 45/130
117/117 [==============================] - 1s - loss: 0.2841 - val_loss: 0.4282
Epoch 46/130
117/117 [==============================] - 1s - loss: 0.3313 - val_loss: 0.4024
Epoch 47/130
117/117 [==============================] - 1s - loss: 0.3067 - val_los

117/117 [==============================] - 1s - loss: 0.1455 - val_loss: 0.1756
Epoch 114/130
117/117 [==============================] - 1s - loss: 0.1474 - val_loss: 0.1729
Epoch 115/130
117/117 [==============================] - 1s - loss: 0.1280 - val_loss: 0.1708
Epoch 116/130
117/117 [==============================] - 1s - loss: 0.1659 - val_loss: 0.1601
Epoch 117/130
117/117 [==============================] - 1s - loss: 0.1664 - val_loss: 0.1977
Epoch 118/130
117/117 [==============================] - 1s - loss: 0.1539 - val_loss: 0.1588
Epoch 119/130
117/117 [==============================] - 1s - loss: 0.1329 - val_loss: 0.1731
Epoch 120/130
117/117 [==============================] - 1s - loss: 0.1517 - val_loss: 0.1708
Epoch 121/130
117/117 [==============================] - 1s - loss: 0.1532 - val_loss: 0.1453
Epoch 122/130
117/117 [==============================] - 1s - loss: 0.1444 - val_loss: 0.1747
Epoch 123/130
117/117 [==============================] - 1s - loss: 0.1405

In [199]:
Model.load_weights('saved_models/weights.best.keras.all.hdf5')

#all_tensors = np.vstack([train_tensors, valid_tensors, test_tensors])
#all_targets = np.vstack([train_targets, valid_targets, test_targets])

melon_predictions = [np.argmax(Model.predict(np.expand_dims(tensor, axis=0))) for tensor in all_tensors]

test_accuracy = 100*np.sum(np.array(melon_predictions)==np.argmax(all_targets, axis=1))/len(melon_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)
np.array(melon_predictions)==np.argmax(all_targets, axis=1)

Test accuracy: 98.2906%


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False])

In [ ]:
import cv2
import matplotlib.pyplot as plt

paths = []

paths.append('images_test_final_app/golden.jpg')
paths.append('images_test_final_app/husky.jpg')
paths.append('images_test_final_app/pastor_alemao.jpg')
paths.append('images_test_final_app/Shih-Tzu.jpg')
paths.append('images_test_final_app/labrador.jpeg')
paths.append('images_test_final_app/pomeranian.jpeg')
paths.append('images_test_final_app/cat.jpeg')
paths.append('images_test_final_app/me.jpg')
paths.append('images_test_final_app/lari_3.jpeg')

fig = plt.figure(figsize=(20,10))
i = 0
for filepath in paths:
    img = cv2.imread(filepath)
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ax = fig.add_subplot(3, 3, i+1, xticks=[], yticks=[])
    ax.imshow(cv_rgb)
    ax.set_title(DogBreedApplication(filepath))
    i += 1


## Transfer learning

In [ ]:
bottleneck_features = np.load('bottleneck_features/DogResnet50Data.npz')
train_RESNET50 = bottleneck_features['train']
test_RESNET50 = bottleneck_features['test']
valid_RESNET50 = bottleneck_features['valid']

In [ ]:
model_RNT = Sequential()
model_RNT.add(GlobalAveragePooling2D(input_shape=train_RESNET50.shape[1:]))
model_RNT.add(Dense(256, activation='relu'))
model_RNT.add(Dropout(0.2))
model_RNT.add(Dense(6, activation='softmax'))
model_RNT.summary()

In [ ]:
model_RNT.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.RESNET50.sgd.hdg5')
model_RNT.fit(train_RESNET50, train_targets,
         validation_data=(valid_RESNET50, valid_targets),
         epochs=40, batch_size=20, verbose=1, callbacks=[checkpointer])

In [ ]:
model_RNT.load_weights('saved_models/weights.best.RESNET50.sgd.hdg5')
RESNET50_predictions = [np.argmax(model_RNT.predict(np.expand_dims(feature, axis=0))) for feature in test_RESNET50]

test_accuracy = 100*np.sum(np.array(RESNET50_predictions)==np.argmax(test_targets, axis=1))/len(RESNET50_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

In [159]:
datagen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(train_tensors)
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.keras.augmentation.hdf5',
                              verbose=1, save_best_only=True)
epochs=100
Model.fit_generator(datagen.flow(train_tensors,train_targets, batch_size=17),
                steps_per_epoch=len(train_tensors) / 17, 
                validation_data=(valid_tensors, valid_targets), epochs=epochs, callbacks=[checkpointer], verbose=1)

Epoch 1/100
4/4 [==============================] - 3s - loss: 1.7914 - val_loss: 1.7801
Epoch 2/100
4/4 [==============================] - 1s - loss: 1.7898 - val_loss: 1.7720
Epoch 3/100
4/4 [==============================] - 1s - loss: 1.7874 - val_loss: 1.8452
Epoch 4/100
4/4 [==============================] - 1s - loss: 1.7812 - val_loss: 2.2220
Epoch 5/100
4/4 [==============================] - 1s - loss: 1.7721 - val_loss: 3.2079
Epoch 6/100
4/4 [==============================] - 1s - loss: 1.7595 - val_loss: 4.6979
Epoch 7/100
4/4 [==============================] - 1s - loss: 1.7337 - val_loss: 7.5701
Epoch 8/100
4/4 [==============================] - 1s - loss: 1.7078 - val_loss: 10.5839
Epoch 9/100
4/4 [==============================] - 1s - loss: 1.6570 - val_loss: 11.7331
Epoch 10/100
4/4 [==============================] - 1s - loss: 1.6280 - val_loss: 11.9134
Epoch 11/100
4/4 [==============================] - 1s - loss: 1.5732 - val_loss: 11.9134
Epoch 12/100
4/4 [========

4/4 [==============================] - 1s - loss: 1.2375 - val_loss: 14.0157
Epoch 44/100
4/4 [==============================] - 1s - loss: 1.2158 - val_loss: 14.0157
Epoch 45/100
4/4 [==============================] - 1s - loss: 1.2820 - val_loss: 14.0157
Epoch 46/100
4/4 [==============================] - 1s - loss: 1.2487 - val_loss: 14.0157
Epoch 47/100
4/4 [==============================] - 1s - loss: 1.2242 - val_loss: 14.0157
Epoch 48/100
4/4 [==============================] - 1s - loss: 1.3684 - val_loss: 14.0157
Epoch 49/100
4/4 [==============================] - 1s - loss: 1.2001 - val_loss: 14.0157
Epoch 50/100
4/4 [==============================] - 1s - loss: 1.2332 - val_loss: 14.0157
Epoch 51/100
4/4 [==============================] - 1s - loss: 1.2735 - val_loss: 14.0157
Epoch 52/100
4/4 [==============================] - 1s - loss: 1.2075 - val_loss: 14.0157
Epoch 53/100
4/4 [==============================] - 1s - loss: 1.2117 - val_loss: 14.0157
Epoch 54/100
4/4 [=====

In [160]:
Model.load_weights('saved_models/weights.best.keras.augmentation.hdf5')

melon_predictions = [np.argmax(Model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

test_accuracy = 100*np.sum(np.array(melon_predictions)==np.argmax(test_targets, axis=1))/len(melon_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 26.9231%
